In [25]:
import numpy as np

from load_nlvr import load_nlvr


# all_image_embeddings = np.concatenate(all_image_embeddings, axis=0)
# np.savetxt("image_embeddings.txt", all_image_embeddings)
# all_text_embeddings = np.concatenate(all_text_embeddings, axis=0)
# np.savetxt("text_embeddings.txt", all_text_embeddings)
# cosine_similarities = np.array(cosine_similarities)
# np.savetxt("cosine_similarities.txt", cosine_similarities)
# attention_entropies = np.concatenate(all_attention_entropies, axis=0)
# np.savetxt("attention_entropies.txt", attention_entropies)

In [26]:
outputs = np.loadtxt("outputs.txt")
all_image_embeddings = np.loadtxt("image_embeddings.txt")
all_text_embeddings = np.loadtxt("text_embeddings.txt")
cosine_similarities = np.loadtxt("cosine_similarities.txt")
attention_entropies = np.loadtxt("attention_entropies.txt")
hidden_states = np.loadtxt("hidden_states.txt")

train_df, val_df, test_df = load_nlvr()

In [27]:
import pandas as pd

# Create a copy of the validation dataframe
combined_df = val_df.copy()

# Make sure the dimensions match
n_samples = len(val_df)
assert outputs.shape[0] == n_samples, "Outputs count doesn't match dataframe length"
assert all_image_embeddings.shape[0] == n_samples, "Image embeddings count doesn't match dataframe length"
assert all_text_embeddings.shape[0] == n_samples, "Text embeddings count doesn't match dataframe length"
assert cosine_similarities.shape[0] == n_samples, "Cosine similarities count doesn't match dataframe length"
assert attention_entropies.shape[0] == n_samples, "Attention entropies count doesn't match dataframe length"

# Add the outputs, image embeddings, and text embeddings to the dataframe
combined_df['outputs'] = outputs

# Add cosine similarity and attention entropy columns
combined_df['cosine_similarity'] = cosine_similarities
combined_df['attention_entropy'] = attention_entropies


# Display the first few rows
print(f"Combined dataframe shape: {combined_df.shape}")
combined_df.head()

Combined dataframe shape: (6982, 8)


,label,sentence,identifier,left,right,outputs,cosine_similarity,attention_entropy
0,False,The right image shows a curving walkway of dar...,dev-850-0-0,data/dev/dev-850-0-img0.png,data/dev/dev-850-0-img1.png,0.0,0.417969,1.703125
1,True,The right image shows a curving walkway of dar...,dev-850-2-0,data/dev/dev-850-2-img0.png,data/dev/dev-850-2-img1.png,1.0,0.419922,1.757812
2,False,The right image shows a curving walkway of dar...,dev-850-3-0,data/dev/dev-850-3-img0.png,data/dev/dev-850-3-img1.png,0.0,0.437500,1.570312
3,True,The right image shows a curving walkway of dar...,dev-850-1-0,data/dev/dev-850-1-img0.png,data/dev/dev-850-1-img1.png,1.0,0.419922,1.664062
4,True,IN at least one image there are at least four ...,dev-850-1-1,data/dev/dev-850-1-img0.png,data/dev/dev-850-1-img1.png,1.0,0.419922,1.656250


In [28]:
# mean cosine similarity
mean_cosine_similarity = combined_df['cosine_similarity'].mean()
print(f"Mean cosine similarity: {mean_cosine_similarity}")
# mean attention entropy
mean_attention_entropy = combined_df['attention_entropy'].mean()
print(f"Mean attention entropy: {mean_attention_entropy}")

Mean cosine similarity: 0.42238971641363504
Mean attention entropy: 1.7003858135204812


In [29]:
# concat image and text embeddings
concatenated_embeddings = np.concatenate((all_image_embeddings, all_text_embeddings), axis=1)
print(f"Concatenated embeddings shape: {concatenated_embeddings.shape}")

Concatenated embeddings shape: (6982, 4096)


In [30]:
from sklearn.metrics import silhouette_score

# Calculate silhouette score
silhouette_avg = silhouette_score(hidden_states, combined_df['label'])
print(f"Silhouette score: {silhouette_avg}")

Silhouette score: 0.10363964381004331


In [31]:
from sklearn.metrics import accuracy_score

# map 0 to False and 1 to True for outputs
combined_df['outputs'] = combined_df['outputs'].map({0: 'False', 1: 'True'})

# Calculate accuracy
accuracy = accuracy_score(combined_df['label'], combined_df['outputs'])

In [32]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.7832999140647379
